# Reading text from data

In [2]:
import nltk
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string 

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('SMSSpamCollection.tsv', sep='\t')
data.columns = ['label', 'body_text']

def count_punch(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punch%'] = data['body_text'].apply(lambda x: count_punch(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)

X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punch%'], pd.DataFrame(X_tfidf.toarray())], axis = 1)
X_features.head()

,body_len,punch%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Applying randomforest classifier  

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassif

In [13]:
rf = RandomForestClassifier(n_jobs = -1)

## Applying cross-validation

In [14]:
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=5)
cross_val_score(rf, X_features, data['label'], cv = k_fold, scoring='accuracy', n_jobs = -1)

array([0.97666068, 0.97666068, 0.97484277, 0.96495957, 0.97124888])

## Explore through Randomforestclassifier hold-out set

In [22]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [23]:
X_train, x_test, Y_train, y_test = train_test_split(X_features, data['label'], test_size = 0.2)

In [24]:
rf_1 = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_1_model = rf_1.fit(X_train, Y_train)


In [25]:
sorted(zip(rf_1_model.feature_importances_, X_train.columns),reverse=True)[0:10]

[(0.05478499313952802, 'body_len'),
 (0.04405023788654852, 1803),
 (0.03374180354983913, 7350),
 (0.030713491010999613, 2031),
 (0.02859005505202307, 3134),
 (0.024510270645311746, 7218),
 (0.021500246288056135, 6285),
 (0.020757593877194513, 4796),
 (0.018923933409308778, 6746),
 (0.018662783552991804, 7027)]

In [27]:
y_pred = rf_1_model.predict(x_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [41]:
print('Precision : {} | Recall : {} | Accuracy : {}'.format(round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision : 1.0 | Recall : 0.595 | Accuracy : 0.946
